In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False: # if True: turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import random


from utils.cityscapesSequence import CitySequence, labels
import utils.dataUtils as dataUtils
from modelclass import DeeplabV3

if gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 8))])

print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.1


In [2]:
DATA_DIR = os.path.join("..", "cityscapes", "allData")

batchSize = 8
imageSize = 256
lrs = [5e-4, 1e-4, 5e-5, 1e-5, 5e-6]
alphas = [0.35, 0.50, 0.75, 1.0, 1.3, 1.4]


##### Dataset


In [3]:
def getData(path="", type="gtFine", extra=False, nSplits=5, seed=0):
    # Load data
    xTrainVal, yTrainVal, xTest, yTest = dataUtils.getXY(path=path, extra=extra, type=type)
    print("TrainVal: ", len(xTrainVal), "Test: ", len(yTest))

    # Split the data in train and validation folds
    return dataUtils.splitTrainValFolds(xTrainVal, yTrainVal, nSplits=nSplits, seed=seed)


def createDatasetObject(trainValFolds, imageSize=imageSize, batchSize=1, remap="binary"):
    for i, (xTrain, yTrain, xVal, yVal) in enumerate(trainValFolds):
        print("Fold", i, "Train: ", len(xTrain), "Val: ", len(xVal))

        dataUtils.dataShuffle(xData=xTrain, yData=yTrain)

        # Create the dataset
        trainDataset = CitySequence(xTrain, yTrain, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=True, verticalFlip=True, brightness=0.1)
        valDataset = CitySequence(xVal, yVal, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=False, verticalFlip=False, brightness=0)

        yield trainDataset, valDataset, trainDataset.nClasses


kf = getData(path=DATA_DIR, type="gtFine", extra=False, nSplits=5, seed=13)
data = createDatasetObject(kf, imageSize=imageSize, batchSize=batchSize, remap="binary")

Getting data from: ..\cityscapes\allData
Getting train data from: ..\cityscapes\allData\leftImg8bit\train\*\*_leftImg8bit.png
Getting val data from: ..\cityscapes\allData\leftImg8bit\val\*\*_leftImg8bit.png
Getting test data from: ..\cityscapes\allData\leftImg8bit\test\*\*_leftImg8bit.png
TrainVal:  3475 Test:  1525


##### Train

In [4]:
losses = {}

for deeplayer in [6, 7, 8, 9, 10, 11, 12]:
    mobileLayers={"shallowLayer": "block_2_project_BN", "deepLayer": "block_" + str(deeplayer) + "_project_BN"}
    
    for lr in lrs:
        for alpha in alphas:
            for i, (trainDataset, valDataset, nClasses) in enumerate(data):
                if alpha > 1. and deeplayer > 10:
                    continue
                
                print("Fold:", i, "learning rate:", lr, "alpha:", alpha)
                
                MODEL_NAME = "deeplabv3plus_fold_" + str(i) + "_lr_" + str(lr) + "_alpha_" + str(alpha) + "_binary_" + str(imageSize) + "x" + str(imageSize) + "_deeplayer_" + str(deeplayer)

                model = DeeplabV3(imageSize=imageSize, nClasses=nClasses, alpha=alpha, modelName=MODEL_NAME, mobileLayers=mobileLayers)
                
                model.run(trainDataset, valDataset, epochs=1000, learningRate=lr, batchSize=batchSize, monitor="val_loss", log=True)

                # Evaluate the model
                losses[MODEL_NAME] = model.evaluate(valDataset)
                print("losses", losses)

Fold 0 Train:  2780 Val:  695
Fold: 0 learning rate: 0.0005 alpha: 0.35
Epoch 1/1000


In [ ]:
losses

{}

##### TFLITE Converter


In [ ]:
from utils.imageUtils import representativeDatasetGen

representativeData = representativeDatasetGen(path="../cityscapes/alldata")

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData, supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS])

MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData)

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)

model.load_weights(MODEL_PATH)

